
# Load data from MySQL to Delta Lake

This notebook shows you how to import data from JDBC MySQL databases into a Delta Lake table using Python.


## Step 1: Connection information

First define some variables to programmatically create these connections.

Replace all the variables in angle brackets `<>` below with the corresponding information.

In [0]:
driver = "org.mariadb.jdbc.Driver"

database_host = "<database-host-url>"
database_port = "3306" # update if you use a non-default port
database_name = "<database-name>"
table = "<table-name>"
user = "<username>"
password = "<password>"

url = f"jdbc:mysql://{database_host}:{database_port}/{database_name}"

print(url)


The full URL printed out above should look something like:

```
jdbc:mysql://localhost:3306/my_database
```

### Check connectivity

Depending on security settings for your MySQL database and Databricks workspace, you may not have the proper ports open to connect.

Replace `<database-host-url>` with the universal locator for your MySQL implementation. If you are using a non-default port, also update the 3306.

Run the cell below to confirm Databricks can reach your MySQL database.

In [0]:
%sh
nc -vz "<database-host-url>" 3306


## Step 2: Reading the data

Now that you've specified the file metadata, you can create a DataFrame. Use an *option* to infer the data schema from the file. You can also explicitly set this to a particular schema if you have one already.

First, create a DataFrame in Python, referencing the variables defined above.

In [0]:
remote_table = (spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("dbtable", table)
    .option("user", user)
    .option("password", password)
    .load()
)

You can view the results of this remote table query.

In [0]:
display(remote_table)


## Step 3: Create a Delta table

The DataFrame defined and displayed above is a temporary connection to the remote database.

To ensure that this data can be accessed by relevant users througout your workspace, save it as a Delta Lake table using the code below.

In [0]:
target_table_name = "<target-schema>.<target-table-name>"
remote_table.write.mode("overwrite").saveAsTable(target_table_name)


This table will persist across cluster sessions, notebooks, and personas throughout your organization.

The code below demonstrates querying this data with Python and SQL.

In [0]:
display(spark.table(target_table_name))

In [0]:
%sql
SELECT * FROM <target-schema>.<target-table-name>